In [6]:
import networkx as nx
import numpy as np
import pandas as pd
from scipy.sparse import csc_matrix
from torch_geometric.utils import k_hop_subgraph
import torch

ModuleNotFoundError: No module named 'scipy'

In [7]:
sample = pd.read_excel('dataset/cora/实验样本.xlsx')
sample

,pid1,pid2,title1,title2,abstract1,abstract2,label1,label2,link
0,101662,101811,Title: Learning from positive data,Title: Programming Research Group A LEARNABILI...,Abstract: Gold showed in 1967 that not even re...,Abstract: This paper compares direct reinforce...,Theory,Theory,Yes
1,22886,1105428,Title: Continuous Case-Based Reasoning,Title: From Theory Refinement to KB Maintenanc...,Abstract: Case-based reasoning systems have tr...,Abstract: Since we consider theory refinement ...,Case_Based,Rule_Learning,No
2,1122642,34257,Title: Word Perfect Corp. LIA: A Location-Inde...,Title: A Self-Adjusting Dynamic Logic Module,Abstract: Most Artificial Neural Networks (ANN...,Abstract: This paper presents an ASOCS (Adapti...,Neural_Networks,Neural_Networks,Yes
3,22886,1120444,Title: Continuous Case-Based Reasoning,Title: Systematic Evaluation of Design Decisio...,Abstract: Case-based reasoning systems have tr...,Abstract: Two important goals in the evaluatio...,Case_Based,Case_Based,Yes
4,10798,20193,Title: Learning physical descriptions from fun...,Title: The Structure-Mapping Engine: Algorithm...,"Abstract: Technical Report 94-07 February 7, 1...",Abstract: This paper describes the Structure-M...,Case_Based,Case_Based,Yes
5,30901,642894,Title: Lookahead and Pathology in Decision Tre...,Title: Rates of convergence of the Hastings an...,Abstract: The standard approach to decision tr...,Abstract: We apply recent results in Markov ch...,Theory,Probabilistic_Methods,No
6,578650,578649,Title: Collective Memory Search 1 Collective M...,Title: Augmenting Collective Adaptation with S...,Abstract: The results reported here empiricall...,Abstract: We have integrated the distributed s...,Genetic_Algorithms,Genetic_Algorithms,Yes
7,10798,91853,Title: Learning physical descriptions from fun...,Title: GIT-CC-92/60 A Model-Based Approach to ...,"Abstract: Technical Report 94-07 February 7, 1...",Abstract: A major issue in case-basedsystems i...,Case_Based,Case_Based,No
8,101662,41666,Title: Learning from positive data,Title: Separating Formal Bounds from Practical...,Abstract: Gold showed in 1967 that not even re...,Abstract: We present a distribution model for ...,Theory,Theory,No
9,141347,1119708,Title: Adapting Crossover in a Genetic Algorithm,Title: On The State of Evolutionary Computation,"Abstract: Traditionally, genetic algorithms ha...",Abstract: In the past few years the evolutiona...,Genetic_Algorithms,Genetic_Algorithms,Yes


## 导入cites

In [8]:
cites_path = 'dataset/cora/cora.cites'
content_path = 'dataset/coracora.content'

edge_index = pd.read_csv(cites_path, delimiter='\t', header=None)
edge_index = torch.tensor(edge_index.values).T

NameError: name 'torch' is not defined

## 抽取子图

In [ ]:
from torch import Tensor

subgraphs = []
for i in range(len(sample)):

    pid1 = sample.loc[i, 'pid1']
    pid2 = sample.loc[i, 'pid2']

    node_idx = (int(pid1), int(pid2))
    node_idx
    # subset, edge_index, mapping, 
    # print(node_idx)
    subset, subgraph, mapping, masked = k_hop_subgraph(node_idx, 1, edge_index, relabel_nodes=False, directed=True)
    
    mask1 = (subgraph[0] != pid2) | (subgraph[1] != pid1)
    mask2 = (subgraph[0] != pid1) | (subgraph[1] != pid2)
    # print(subgraph)
    # print(mask2 & mask1)
    subgraph = subgraph[:, mask1 & mask2].tolist()
    subgraphs.append(subgraph)
    
sample['subgraph_1hop'] = subgraphs
sample = sample[['pid1', 'pid2', 'title1', 'title2', 'abstract1', 'abstract2', 'label1', 'label2', 'link', 'subgraph_1hop']]
sample

## 关联title label

In [ ]:
cora_text = pd.read_csv('dataset/cora/cora_text.tsv', delimiter='\t')
cora_text

# 关联标题
subgraphs_title = []
for i in range(len(sample)):
    subgraph = sample.loc[i, 'subgraph_1hop']
    subgraph_title = [[], []]
    for j in range(2):
        for cite in subgraph[j]:
            title = cora_text[cora_text.pid == int(cite)].title.values[0]
            
            label = cora_text[cora_text.pid == int(cite)].label.values[0]
            subgraph_title[j].append(title + ', category:' + label)
    subgraphs_title.append(subgraph_title)

sample['subgraph_1hop_title'] = subgraphs_title
sample

In [ ]:
# subgraph, subgraph1

import networkx as nx
import matplotlib.pyplot as plt

# 导入 Cora 数据集或创建一个示例图
for i in range(20):
    subgraph = sample.loc[i, 'subgraph_1hop']
    # subgraph1 = sample1.loc[i, 'subgraph_1hop']
    
    cora_graph = nx.Graph()
    cora_graph.add_edges_from(torch.tensor(subgraph).numpy().T)
    # cora_graph.add_edges_from(sg)
    # 可视化图
    pos = nx.spring_layout(cora_graph)  # 定义节点布局
    nx.draw(cora_graph, pos, with_labels=True, node_color='lightblue', node_size=300, edge_color='gray')  # 绘制图
    plt.savefig(f'2hop/{i}_masked.png')
    
    # # 清空画布
    plt.cla()
    
    # cora_graph = nx.Graph()
    # cora_graph.add_edges_from(torch.tensor(subgraph1).numpy().T)
    # # cora_graph.add_edges_from(sg)
    # # 可视化图
    # pos = nx.spring_layout(cora_graph)  # 定义节点布局
    # nx.draw(cora_graph, pos, with_labels=True, node_color='lightblue', node_size=300, edge_color='gray')  # 绘制图
    # plt.savefig(f'2hop/{i}.png')
    
    # # 清空画布
    # plt.cla()
# plt.show()  # 显示图形